In [46]:
from collections import OrderedDict
import copy
import requests
import json
import copy
import random
import operator
json_new = 'SQUAD_DIR/ensemble_method1.json'
prediction_base = 'SQUAD_DIR/squad2.0_base_predictions.json'
general_1 = 'SQUAD_DIR/squad2.0_base_nbest_predictions-jun1.json'
general_2 = 'SQUAD_DIR/squad_result_2.1-nbest_predictions_Rhett1.json'
general_3 = 'SQUAD_DIR/squad2.0_base_nbest_predictions_jane1.json'
special_1 = 'SQUAD_DIR/squad_result_2.3_batchsize32_30%noise_nbest_predictions.json'
special_2 = 'SQUAD_DIR/squad2.0_base_result_15%noise_nbest_predictions.json'
special_3 = 'SQUAD_DIR/squad_result_batch32_0%noise_nbest_predictions_jun2.json'
file = open(json_new, 'w')
with open(prediction_base) as s:
    with open(general_1) as f:
        with open(general_2) as g:
            with open(general_3) as h:
                with open(special_1) as x:
                    with open(special_2) as y:
                        with open(special_3) as z:
                            g1 = json.load(f)
                            g2 = json.load(g)
                            g3 = json.load(h)
                            s1 = json.load(x)
                            s2 = json.load(y)
                            s3 = json.load(z)
                            pred = json.load(s)

In [47]:
len(pred) #11873 predictions 
list(pred.keys())[0]


'56ddde6b9a695914005b9628'

In [48]:
#无答案的给general model决策
# g是general model, s是specialized model。
g_list = [g1, g2, g3]
s_list = [s3, s3, s3]
#s1~30%, s2~15%, s3~0%
no_answer_idList = []
n = len(g_list)

for ID in list(pred.keys()):
    count_noAnswer = 0
    count_Answer = 0
    for g in g_list:
        if g[ID][0]['text'] == '':
            count_noAnswer += 1
        else: 
            count_Answer += 1
#     print ('no answer: ', count_noAnswer)
#     print ('answer: ', count_Answer)
    if count_noAnswer > count_Answer:
        pred[ID] = ''
        no_answer_idList.append(ID)

In [49]:
#方法1:有答案的，按照special model概率相加投票
for ID in list(g1.keys()):
    if ID in no_answer_idList: 
        continue
    else:
        v1 = OrderedDict()
        v2 = OrderedDict()
        v3 = OrderedDict()
#         print (ID)
    #     for v_unit in v_list:
        for rank in range(5):
            v1[s1[ID][rank]['text']] = s1[ID][rank]['probability']
            v2[s2[ID][rank]['text']] = s2[ID][rank]['probability']
            v3[s3[ID][rank]['text']] = s3[ID][rank]['probability']
        v_list = [v1, v2, v3]
        V = v1.copy()
        V.update(v2)
        V.update(v3)
        for key in V.keys():
            V[key] = 0
        for key in V.keys():
            for v0 in v_list:
                if key in v0:
                    V[key] = float(V[key]) + float(v0[key])
        R = sorted(V.items(),key = operator.itemgetter(1))
        pred[ID] = R[-1][0]

In [8]:
# #方法二，有答案的，直接按照special model排名最高的答案投票
# for ID in list(g1.keys()):
#     if ID in no_answer_idList: 
#         continue
#     else:  
#         v1 = OrderedDict()
#         v2 = OrderedDict()
#         v3 = OrderedDict()
# #         print (ID)
#     #     for v_unit in v_list:
# #         for rank in range(10):
#         rank = 0
#         v1[s1[ID][rank]['text']] = s1[ID][rank]['probability']
#         v2[s2[ID][rank]['text']] = s2[ID][rank]['probability']
#         v3[s3[ID][rank]['text']] = s3[ID][rank]['probability']
#         v_list = [v1, v2, v3]
#         V = v1.copy()
#         V.update(v2)
#         V.update(v3)
#         for key in V.keys():
#             V[key] = 0
#         for key in V.keys():
#             for v0 in v_list:
#                 if key in v0:
#                     V[key] = float(V[key]) + 1
#         sorted(V.items(),key = operator.itemgetter(1))
#         pred[ID] = list(V.keys())[0]

In [50]:
#保存
pred = json.dumps(pred)
file.write(pred)
file.close()